In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
data_train_path="/content/drive/MyDrive/BallData/train"
data_test_path="/content/drive/MyDrive/BallData/test"

batch_size=4
img_size=224
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001,decay = 1e-4)
loss="categorical_crossentropy"
epochs=10
onnx_path="/content/drive/MyDrive/BallOnnxModel/model.onnx"


In [ ]:
image_generator_test = ImageDataGenerator(rescale=1./255,)    
data_train = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_train_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )
data_test = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_test_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )

Found 172 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
print("Train Data",data_train.class_indices)
print("Test Data",data_test.class_indices)



Train Data {'colored_beachball': 0, 'football': 1}
Test Data {'colored_beachball': 0, 'football': 1}


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization,concatenate,Conv2DTranspose,Dropout

inputs=Input(shape=(224,224,3),name="input")
    
x = Conv2D(16,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(256,(3,3),activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x=tf.keras.layers.GlobalMaxPool2D()(x)

x=tf.keras.layers.Dense(128,activation="relu")(x)
x=tf.keras.layers.Dense(len(data_train.class_indices),activation="softmax",name="output")(x)

model=tf.keras.Model(inputs=inputs,outputs=x)
model.summary()

model.compile(optimizer =optimizer,loss = loss,  metrics = ["accuracy"])


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 56, 56, 64)        1849

In [ ]:
callbacks=[]
callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',factor = 0.1,patience = 2,min=1e-6,verbose=1))
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor="loss",patience=4,restore_best_weights=True))

In [ ]:
model.fit(data_train,validation_data=data_test,  epochs = 10,callbacks=callbacks)


In [ ]:
from PIL import Image
import numpy as np

img = Image.open("/content/drive/MyDrive/BallData/test/football/football22.png").convert('RGB').resize((img_size,img_size))
img=np.asarray(img)
predicted=model.predict(np.expand_dims(img,axis=0))
predicted
list(data_train.class_indices.keys())[np.argmax(predicted)]

1/1 [==============================] - 0s 72ms/step


'football'

In [ ]:
predicted

array([[0., 1.]], dtype=float32)

In [ ]:
import os
save_model_path="/content/saved_model"
if not os.path.isdir(save_model_path):
      os.makedirs(save_model_path)

model.save(save_model_path)

In [ ]:
!pip install onnxruntime
!pip install onnx
!pip install -U tf2onnx



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have pr

In [ ]:
!python -m tf2onnx.convert --saved-model /content/saved_model --opset 9 --output {onnx_path}



/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-02-01 11:36:20,900 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2023-02-01 11:36:21,234 - INFO - Signatures found in model: [serving_default].
2023-02-01 11:36:21,234 - WARNING - '--signature_def' not specified, using first signature: serving_default
2023-02-01 11:36:21,234 - INFO - Output names: ['output']
2023-02-01 11:36:21,355 - INFO - Using tensorflow=2.9.2, onnx=1.13.0, tf2onnx=1.13.0/2c1db5
2023-02-01 11:36:21,355 - INFO - Using opset <onnx, 9>
2023-02-01 11:36:21,454 - INFO - Computed 0 values for constant folding
2023-02-01 11:36:21,536 - INFO - Optimizing ONNX model
2023-02-01 11:36:21,605 - INFO - After optimization: Const -1 (15->14), GlobalMaxPool +1 (0->1), Identity -2 (2->0), ReduceMax -1 (1->0), Squeeze +1 

In [ ]:
import numpy as np
import onnxruntime as ort

img = Image.open("/content/WhatsApp Image 2023-02-01 at 14.12.18 (1).jpeg").convert('RGB').resize((224,224))
img=np.expand_dims(np.asarray(img, dtype="float32"),axis=0)
img=img/255

sess_ort = ort.InferenceSession(onnx_path,providers=ort.get_available_providers())

outputs = sess_ort.run(None, {sess_ort.get_inputs()[0].name: img})
print(outputs)


[array([[0.9725016 , 0.02749839]], dtype=float32)]
